# Viewing ase structure using `imolecule`

In [2]:
import imolecule
import numpy as np

In [3]:
def atoms_to_json(aseatoms, infer_bonds=True):
    '''
    Convert ASE Atoms isntance into the json format compatible with
    
    Args:
        aseatoms : ase.Atoms
            Instance of Atoms from ase package
        infer_bonds : bool
            If `True` bonds will be inferred using openbabel

    Returns:
        mol : dist
            A dictionary with the json format of the molecule
    '''

    import pybel
    ob = pybel.ob
    
    obmol = ob.OBMol()
    obmol.BeginModify()
    
    for atom in aseatoms:
        obatom = obmol.NewAtom()
        obatom.SetAtomicNum(int(atom.number))
        obatom.SetVector(*atom.position.tolist())

    # If there is no bond data, try to infer them
    if infer_bonds:
        obmol.ConnectTheDots()
        obmol.PerceiveBondOrders()
    
    # Check for unit cell data
    if any(aseatoms.pbc):
        uc = ob.OBUnitCell()
        uc.SetData(*(ob.vector3(*v) for v in aseatoms.get_cell()))
        uc.SetSpaceGroup('P1')
        obmol.CloneData(uc)
    obmol.EndModify()

    mol = pybel.Molecule(obmol)

    return imolecule.format_converter.pybel_to_json(mol)

# Molecules

In [5]:
from ase.build import molecule

In [6]:
benzene = molecule('C6H6')
benzene_json = atoms_to_json(benzene)
imolecule.draw(benzene_json, format='json', size=(800, 500))

In [9]:
benzene.set_cell(np.eye(3) * 14.0)
benzene.set_pbc(True)
benzene_json = atoms_to_json(benzene)
imolecule.draw(benzene_json, format='json', size=(800, 500), drawing_type='space filling')

# Zeolite

In [11]:
def from_string(string, fmt=None):
    'Instantiate Atoms object from a string with specific format'

    import ase.io
    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+') as ntf:
        ntf.write(string)
        ntf.seek(0)
        atoms = ase.io.read(ntf, format=fmt)

    return atoms

In [12]:
aficif ='''data_AFI
#**************************************************************************
#
# CIF taken from the IZA-SC Database of Zeolite Structures
# Ch. Baerlocher and L.B. McCusker
# Database of Zeolite Structures: http://www.iza-structure.org/databases/ 
#
# The atom coordinates and the cell parameters were optimized with DLS76
# assuming a pure SiO2 composition.
#
#**************************************************************************


_cell_length_a                  13.8270(0)
_cell_length_b                  13.8270(0)
_cell_length_c                   8.5800(0)
_cell_angle_alpha               90.0000(0)
_cell_angle_beta                90.0000(0)
_cell_angle_gamma              120.0000(0)

_symmetry_space_group_name_H-M     'P 6/m c c'
_symmetry_Int_Tables_number         192
_symmetry_cell_setting             hexagonal


loop_
_symmetry_equiv_pos_as_xyz
'+x,+y,+z'
'-y,+x-y,+z'
'-x+y,-x,+z'
'-x,-y,+z'
'+y,-x+y,+z'
'+x-y,+x,+z'
'-y,-x,1/2+z'
'-x+y,+y,1/2+z'
'+x,+x-y,1/2+z'
'+y,+x,1/2+z'
'+x-y,-y,1/2+z'
'-x,-x+y,1/2+z'
'-x,-y,-z'
'+y,-x+y,-z'
'+x-y,+x,-z'
'+x,+y,-z'
'-y,+x-y,-z'
'-x+y,-x,-z'
'+y,+x,1/2-z'
'+x-y,-y,1/2-z'
'-x,-x+y,1/2-z'
'-y,-x,1/2-z'
'-x+y,+y,1/2-z'
'+x,+x-y,1/2-z'

loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
    O1    O      0.4565    0.3333    0.0000
    O2    O      0.3693    0.3693    0.2500
    O3    O      0.4200    0.2100    0.2500
    O4    O      0.5798    0.4202    0.2500
    T1    Si     0.4565    0.3334    0.1874

'''

In [13]:
afi = from_string(aficif, fmt='cif')
afi

Atoms(symbols='O48Si24', pbc=True, cell=[[13.827, 0.0, 0.0], [-6.913499999999997, 11.974533258127433, 0.0], [5.253734768342146e-16, 9.0997355482597e-16, 8.58]])

In [15]:
afi_json = atoms_to_json(afi)
imolecule.draw(afi_json, format='json', size=(800, 500))

In [18]:
from ase.build import cut
afi_sc = cut(afi, a=(1, 0, 0), b=(0, 1, 0), c=(0, 0, 2), origo=(0.5, 0.5, 0.0))

In [19]:
afi_sc_json = atoms_to_json(afi_sc)
imolecule.draw(afi_sc_json, format='json', size=(800, 500))

## MOF

In [17]:
def from_url(url, fmt=None):

    if fmt.lower() == 'json':
        import json

        with urllib.request.urlopen(url) as response:
            contents = response.read()
        atoms = json.loads(contents)

In [21]:
# read the UiO-66 structure in the json format from imolecule repo
import urllib
import json

url ='https://raw.githubusercontent.com/patrickfuller/imolecule/master/examples/data/uio_66.json'
with urllib.request.urlopen(url) as response:
    uiostr = response.read()
uio66 = json.loads(uiostr)

In [22]:
imolecule.draw(uio66, format='json', size=(800, 800))

In [23]:
%version_information numpy, ase, imolecule

Software versions
Python 3.6.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 6.0.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.7
numpy 1.12.1
ase 3.13.0
imolecule 0.1.13
Thu May 04 17:26:55 2017 CEST